# 外Azure OpenAI + Logic Apps で天気予報を取得する (実行編)
## アシスタントと Functions の実行

この例では、天気のデータを使って今日の天気をAzure Open AI に回答させます。

In [4]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
from typing import Optional

load_dotenv(verbose=True)

    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [5]:
def poll_run_till_completion(
    client: AzureOpenAI,
    thread_id: str,
    run_id: str,
    available_functions: dict,
    verbose: bool,
    max_steps: int = 10,
    wait: int = 3,
) -> None:
    import time
    """
    Poll a run until it is completed or failed or exceeds a certain number of iterations (MAX_STEPS)
    with a preset wait in between polls

     client: Azure OpenAI client
     thread_id: Thread ID
     run_id: Run ID
     assistant_id: Assistant ID
     verbose: Print verbose output
     max_steps: Maximum number of steps to poll
     wait: Wait time in seconds between polls

    """

    if (client is None and thread_id is None) or run_id is None:
        print("Client, Thread ID and Run ID are required.")
        return
    try:
        cnt = 0
        while cnt < max_steps:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if verbose:
                print("Poll {}: {}".format(cnt, run.status))
            cnt += 1
            if run.status == "requires_action":
                tool_responses = []
                if (
                    run.required_action.type == "submit_tool_outputs"
                    and run.required_action.submit_tool_outputs.tool_calls is not None
                ):
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls

                    for call in tool_calls:
                        if call.type == "function":
                            if call.function.name not in available_functions:
                                raise Exception("Function requested by the model does not exist")
                            function_to_call = available_functions[call.function.name]
                            tool_response = function_to_call(**json.loads(call.function.arguments))
                            tool_responses.append({"tool_call_id": call.id, "output": tool_response})

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run.id, tool_outputs=tool_responses
                )
            if run.status == "failed":
                print("Run failed.")
                break
            if run.status == "completed":
                break
            time.sleep(wait)

    except Exception as e:
        print(e)

In [6]:
def retrieve_and_print_messages(
    client: AzureOpenAI, thread_id: str, verbose: bool, out_dir: Optional[str] = None
) -> any:
    """
    Retrieve a list of messages in a thread and print it out with the query and response

     client: OpenAI client
     thread_id: Thread ID
     verbose: Print verbose output
     out_dir: Output directory to save images
    @return: Messages object

    """
    from pathlib import Path

    if client is None and thread_id is None:
        print("Client and Thread ID are required.")
        return None
    try:
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        display_role = {"user": "User query", "assistant": "Assistant response"}

        prev_role = None

        if verbose:
            print("\n\nCONVERSATION:")
        for md in reversed(messages.data):
            if prev_role == "assistant" and md.role == "user" and verbose:
                print("------ \n")

            for mc in md.content:
                # Check if valid text field is present in the mc object
                if mc.type == "text":
                    txt_val = mc.text.value
                # Check if valid image field is present in the mc object
                elif mc.type == "image_file":
                    image_data = client.files.content(mc.image_file.file_id)
                    if out_dir is not None:
                        out_dir_path = Path(out_dir)
                        if out_dir_path.exists():
                            image_path = out_dir_path / (mc.image_file.file_id + ".png")
                            with image_path.open("wb") as f:
                                f.write(image_data.read())

                if verbose:
                    if prev_role == md.role:
                        print(txt_val)
                    else:
                        print("{}:\n{}".format(display_role[md.role], txt_val))
            prev_role = md.role
        return messages
    except Exception as e:
        print(e)
        return None

In [7]:
def get_weather(location):
    import urllib.request
    try:
        url = os.getenv("GET_WEATHER_URL")
    
        data = {
            'location': location
        }
    
        headers = {
            'Content-Type': 'application/json'
        }
        
        req = urllib.request.Request(url, json.dumps(data).encode(), headers)
        body = None
        with urllib.request.urlopen(req) as res:
            body = json.load(res)
        
        print(type(body))
        weather_condition = body['responses']['daily']['day']['cap']
        temperature_hi = body['responses']['daily']['tempHi']
        temperature_lo = body['responses']['daily']['tempLo']
        return f"""Here is some information about the weather in {location}:
            - The weather is: {weather_condition}.
            - The highest temperature is: {temperature_hi} degrees Celsius.
            - The Lowest Temperatures is: {temperature_lo} degrees Celsius.
        """
    except Exception as e:
        print(e)

In [8]:
get_weather(location="北区田端")

<class 'dict'>


'Here is some information about the weather in 北区田端:\n            - The weather is: Mostly sunny.\n            - The highest temperature is: 36.0 degrees Celsius.\n            - The Lowest Temperatures is: 27.0 degrees Celsius.\n        '

#### アシスタントの作成

In [9]:
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes forecasting the weather based on data."
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. To forecast the weather, use get_weather defined in the tool."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[
        {
            "type": "code_interpreter"
        },
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "場所の検索クエリ。 都市、地域、都道府県、国、ランドマーク、郵便番号、緯度、経度を入力できます"
                        }
                    },
                    "required": [
                        "location"
                    ]
                }
            }
        }
    ],
    model=f"{os.getenv("AZURE_OPENAI_MODEL_NAME")}" #You must replace this value with the deployment name for your model.
)

In [10]:
print(assistant)

Assistant(id='asst_RM4iB6ai1UWy0JGfCQjeb7Lj', created_at=1723618961, description=None, instructions='You are a helpful AI assistant who makes forecasting the weather based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. To forecast the weather, use get_weather defined in the tool.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.', metadata={}, model='gpt-4o', name='Data Visualization', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter'), FunctionTool(function=FunctionDefinition(name='get_weather', description=None, parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': '場所の検索クエリ。 都市、地域、都道府県、国、ランドマーク、郵便番号、

#### アシスタントの内容の出力

In [11]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_RM4iB6ai1UWy0JGfCQjeb7Lj",
  "created_at": 1723618961,
  "description": null,
  "instructions": "You are a helpful AI assistant who makes forecasting the weather based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. To forecast the weather, use get_weather defined in the tool.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4o",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    },
    {
      "function": {
        "name": "get_weather",
        "description": null,
        "parameters": {
          "type": "object",
          "properties": {
    

#### スレッドを作成してみます

In [12]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_Et4nVn4fxxbnrBpa9M9ppnag', created_at=1723618969, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


#### スレッドに追加する最初のユーザーの質問を作成

In [13]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="現在の天気を答えてください"
)

#### スレッド メッセージを一覧表示する

In [14]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_UHDnSoTfZ6wdDUP7RjKkMFtr",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "今日の日本の帰宅の天気を答えてください"
          },
          "type": "text"
        }
      ],
      "created_at": 1723618971,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_Et4nVn4fxxbnrBpa9M9ppnag"
    }
  ],
  "object": "list",
  "first_id": "msg_UHDnSoTfZ6wdDUP7RjKkMFtr",
  "last_id": "msg_UHDnSoTfZ6wdDUP7RjKkMFtr",
  "has_more": false
}


#### スレッドを実行する

In [15]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="今日の東京都田端の天気を教えてください。" #You can optionally provide new instructions but these will override the default instructions
)

In [17]:
available_functions = {"get_weather": get_weather}
verbose_output = True

In [18]:
poll_run_till_completion(
    client=client, thread_id=thread.id, run_id=run.id, available_functions=available_functions, verbose=verbose_output
)

Poll 0: requires_action
<class 'dict'>
Poll 1: completed


In [19]:
messages = retrieve_and_print_messages(client=client, thread_id=thread.id, verbose=verbose_output)



CONVERSATION:
User query:
今日の日本の帰宅の天気を答えてください
Assistant response:
今日の東京都田端の天気は、
- 天気: 主に晴れ
- 最高気温: 36.0度
- 最低気温: 27.0度

です。


#### スレッド状態を取得する

In [ ]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

#### 都度実行では大変なので、ループで状態監視をします

In [ ]:
def check_status(thread, run):
    import time
    from IPython.display import clear_output
    start_time = time.time()

    status = run.status
    
    while status not in ["completed", "cancelled", "expired", "failed"]:
        time.sleep(5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
        print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
        status = run.status
        print(f'Status: {status}')
        clear_output(wait=True)
    
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    ) 
    
    print(f'Status: {status}')
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    print(messages.model_dump_json(indent=2))

In [ ]:
check_status(thread=thread, run=run)

#### 実行後のスレッド メッセージを一覧表示する

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

#### ファイル ID を取得する

モデルに正弦波の画像を生成させるように要求しました。 画像をダウンロードするには、まず画像ファイル ID を取得する必要があります。


In [ ]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD

#### イメージをダウンロードする

In [ ]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

<p>ダウンロードしたら、画像をローカルで開きます。</p>

In [ ]:
from PIL import Image
from IPython.display import display

# Display the image in the default image viewer
image = Image.open("sinewave.png")
#image.show()
display(image)

### スレッドでフォローアップの質問をする

アシスタントがこちらの指示に完全には従わず、応答のテキスト部分に実行されたコードが含まれていなかったため、明示的にその情報を要求してみます。

In [ ]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

もう一度実行して、スレッドの状態を取得する必要があります。

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

check_status(thread=thread, run=run)

実行状態が完了に達したら、スレッド内のメッセージを再度一覧表示します。これには、最新の質問への応答が含まれているはずです。

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

最新の質問に対する応答のみを抽出するには、以下を行います。

In [ ]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

【演習】正弦波の視覚化を生成するために使ったコードを張り付けて実行してみましょう

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#...

#### ダーク モード
最後の質問をスレッドに追加して、コード インタープリターがグラフをダーク モードに切り替えられるかどうかを確認してみましょう。

In [ ]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I prefer visualizations in darkmode can you change the colors to make a darkmode version of this visualization."
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
check_status(thread=thread, run=run)

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

新しい画像ファイル ID を抽出し、画像をダウンロードして表示します。

In [ ]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id'] # index numbers can vary if you have had a different conversation over the course of the thread.

print(image_file_id)

content = client.files.content(image_file_id)
image= content.write_to_file("dark_sine.png")

# Display the image in the default image viewer
image = Image.open("dark_sine.png")
#image.show()
display(image)

[READMEに戻る](./README.markdown)